# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
path = os.path.join("..","WeatherPy", "Weather Check2.csv")
weather_df = pd.read_csv(path)
weather_df = weather_df[['City', 'Country','Temperature (F)', 'Humidity (%)', 'Cloudiness (%)', 'Wind Speed (mph)', 'Lat', 'Lon']]
weather_df

,City,Country,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Lat,Lon
0,Båtsfjord,NO,38.39,77.0,51.0,25.66,70.6345,29.7185
1,Tiksi,RU,36.63,71.0,100.0,9.69,71.6872,128.8694
2,Hithadhoo,MV,83.26,73.0,100.0,12.26,-0.6000,73.0833
3,Calama,CL,53.98,27.0,55.0,0.94,-22.4667,-68.9333
4,Punta Arenas,CL,51.91,66.0,75.0,17.27,-53.1500,-70.9167
...,...,...,...,...,...,...,...,...
570,Lodeynoye Pole,RU,34.90,86.0,100.0,7.65,60.7260,33.5531
571,Cockburn Town,TC,83.03,79.0,83.0,18.84,21.4612,-71.1419
572,jahrom,ir,NaN,NaN,NaN,NaN,NaN,NaN
573,Ostashkov,RU,36.88,98.0,100.0,10.29,57.1467,33.1075


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
weather_df_complete = weather_df.dropna()
locations = weather_df_complete[["Lat", "Lon"]].astype(float)
humidity = weather_df_complete['Humidity (%)'].astype(float)

In [6]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

In [7]:
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
ideal_df = weather_df_complete[weather_df_complete['Temperature (F)']<80] 
ideal_df = ideal_df[ideal_df['Temperature (F)']>70]
ideal_df = ideal_df[ideal_df['Wind Speed (mph)']<10]
ideal_df = ideal_df[ideal_df['Cloudiness (%)']==0]
print(len(ideal_df))
ideal_df = ideal_df.drop_duplicates()
ideal_df = ideal_df.reset_index()
ideal_df = ideal_df[['City','Country','Temperature (F)', 'Humidity (%)', 'Cloudiness (%)', 'Wind Speed (mph)', 'Lat', 'Lon']]
ideal_df

12


,City,Country,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Lat,Lon
0,Alexandria,EG,78.76,83.0,0.0,5.75,31.2156,29.9553
1,Maceió,BR,72.84,94.0,0.0,3.44,-9.6658,-35.7353
2,Abu Samrah,SY,73.24,46.0,0.0,5.19,35.3029,37.1841
3,Tongzi,CN,78.39,73.0,0.0,2.44,28.7829,105.0877
4,Buraidah,SA,75.65,19.0,0.0,3.74,26.3260,43.9750
5,Camalú,MX,70.20,74.0,0.0,4.92,30.8500,-116.0667
6,Dalbandin,PK,76.57,16.0,0.0,2.30,28.8947,64.4101
7,Obama,JP,79.39,57.0,0.0,8.77,35.4958,135.7460
8,Pishin,PK,70.09,23.0,0.0,1.16,30.5803,66.9961
9,Cayenne,GF,77.04,94.0,0.0,1.14,4.9333,-52.3333


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df =ideal_df[['City', 'Country','Lat', 'Lon']].copy()
hotel_df['Hotel Name'] = ''

In [15]:
for city in hotel_df['City']:
    print(hotel_df[hotel_df['City']==city]['Lat'])
    break
    target_coordinates = f" {hotel_df[hotel_df['City']==city]['Lat']}, {hotel_df[hotel_df['City']==city]['Lon']} "
    print(target_coordinates)
    break
    target_radius = 5000
    target_type = "lodging"
    keyword = 'hotel'
    # set up a parameters dictionary
    params = {"location": target_coordinates,
              "radius": target_radius, 
              "type": target_type,
              "key": g_key}
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    # convert response to json
    hotel_data = response.json()
    pprint(hotel_data)#["results"][0]["name"])
    #print(hotel_data["results"][0]["vicinity"])

# Print the json (pretty printed)
#print(json.dumps(places_data, indent=4, sort_keys=True))

0    31.2156
Name: Lat, dtype: float64


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [18]:
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…